In [1]:
import pandas as pd
from structured_agent import *
from BanditAnalysisFunctions import *
from cprobs_functions import *
%matplotlib qt



In [5]:
structured?

Signature:
structured(
    trials,
    sessions,
    probs,
    coupling,
    policy,
    params,
    window,
    output,
    plot,
)
Docstring:
probs; 'all' or specify as percentages
coupling parameter defines degree of structure (0=<c=<1)
policy: 'softmax' or 'epsilon-greedy'
params in order: alpha, epsilon, tau
window; window size for moving average
output; 'reward rate' / 'soft regret' / 'hard regret'
plot; Bool
store choice, reward, trial#, session#, rewprob1, rewprob2, better rewprob
File:      c:\users\dlab\downloads\autotrainermodular\structured_agent.py
Type:      function

In [11]:
df_str = structured(150, 100, 'all', 1, 'softmax', [0.1, 0.2, 0.3], 5, 'hard regret', False)

In [10]:
df_ustr_env = unstructured(150, 100, 'all', 1, 'softmax', [0.1, 0.2, 0.3], 5, 'hard regret', False)

In [16]:
df_ustr = structured(150, 50000, 'all', 0, 'softmax', [0.1, 0.2, 0.3], 5, 'hard regret', False)

In [18]:
df_str.to_csv('model_str.csv')
df_ustr.to_csv('model_ustr.csv')

In [14]:
df_str = pd.read_csv('model_str.csv')
df_ustr = pd.read_csv('model_ustr.csv')

#### Regret

In [13]:

#inter-animal reg:
x_lim = 149
n_sess = 50
df_list = [subset(df_ustr_env), df_str]
fig, ax = plt.subplots(layout= 'tight')
color = ['r', 'g']
c = 0
for df in df_list:
    ax.plot(range(x_lim), pd.DataFrame(hard_regret(df, x_lim, x_lim)).rolling(1).mean(numeric_only = True), color[c]) 
    c += 1
    ax.set(xlabel = 'trial#', ylabel = 'hard regret')
    plt.title('Simulation: str task\, alpha = 0.1, temp = 0.3')
    plt.legend(['unstructured', 'structured'])
    ax.set(ylim = (0, 0.7))
    sns.despine()
    plt.show()
    

15
100


In [3]:
plt.figure()

<Figure size 640x480 with 0 Axes>

In [12]:
df_str

,trial#,session#,port,reward,rewprobfull1,rewprobfull2,rw
0,1,1,1,0,70,30,70
1,2,1,2,0,70,30,30
2,3,1,1,0,70,30,70
3,4,1,2,0,70,30,30
4,5,1,1,1,70,30,70
...,...,...,...,...,...,...,...
79995,96,800,1,0,70,30,70
79996,97,800,1,0,70,30,70
79997,98,800,1,0,70,30,70
79998,99,800,1,1,70,30,70


#### RR

In [15]:
#reward rate
df_list = [df_ustr, df_str]
window = 5
fig, ax = plt.subplots(layout = 'tight')
c = 0
color = ['r', 'g']
for df in df_list:
    sess_list = []
    for i in range(max(pd.unique(df['session#']))):
        sess_list.append(df[df['session#'] == (i+1)]['reward'].reset_index(drop = True))
    df_rewards = pd.concat(sess_list, axis = 1)
    rewards_mav = df_rewards.mean(axis = 1).rolling(window, center = True).mean()
    
    ax.plot(range(len(rewards_mav)), rewards_mav, color[c])
    ax.set(xlabel = 'trial#', ylabel = 'reward rate', ylim = [0.5, 0.7])
    sns.despine()
    c += 1

#### Switching

In [60]:
def switchProb_trial(df, sess_len, include_sess, window = 20, animal = '', start = 'all'):
    '''
    return switch probability as a function of trial number
    inputs:
    sessdf: original animal df
    sess_len: number of initial trials to be considered
    include_sess: last n sessions to be considered
    window: moving window for pd.rolling() | Default = 20
    subset: bool, whether to subset str sessions from unstr dataframes | Default = True || REMOVED
    animal: name of animal 
    start: whether the animal started the session with a correct or incorrect choice ('correct' /'incorrect'/'all') | Default = 'all'
    outputs:
    array of switch probs averaged across sessions
    '''
    print('no. of sess before subsetting: ', len(pd.unique(df['session#'])))
    
    # if subset == True:
    #     df = df[((df['rewprobfull1']==10)&(df['rewprobfull2']==90))|((df['rewprobfull1']==20)&(df['rewprobfull2']==80))|((df['rewprobfull1']==30)&(df['rewprobfull2']==70))|((df['rewprobfull1']==40)&(df['rewprobfull2']==60|((df['rewprobfull1']==90)&(df['rewprobfull2']==10))|((df['rewprobfull1']==80)&(df['rewprobfull2']==20))|((df['rewprobfull1']==70)&(df['rewprobfull2']==30))|((df['rewprobfull1']==60)&(df['rewprobfull2']==40))))]    
    # print(df.groupby('session#').mean(numeric_only = True))
    df = df[['trial#', 'session#','port', 'reward', 'rewprobfull1', 'rewprobfull2', 'rw']] 
    sessions = pd.unique(df['session#'])
    print('No. of sess after subsetting: ', len(sessions))
    # print(sessions)
    # print(np.max(sessions))
    exclude = np.max(sessions)- include_sess ##### this assumes no subsetting
    # print('ex: ', exclude)
    # print('dfb: ', df)
    # df = df[df['session#']>exclude] #only keep n sessions specified in include_sess
    # print('dfa: ', df)
    switches = abs(np.diff(df['port'].tolist()))
    switches = np.insert(switches, 0, 0, axis = 0) #convert choices to 0/1:stay/switch encoding and add a 0 to the first row
    df.insert(2, 'switch', switches ) 

    df = df.groupby('session#').filter(lambda x: len(x['trial#'])>sess_len) # filter sessions to include only those longer than sess_len
    df = df.groupby('session#').head(sess_len)
    if start == 'correct': 
        df = df.groupby('session#').filter(lambda x: x.iloc[0]['rw'] > 50 )
    elif start == 'incorrect':
        df = df.groupby('session#').filter(lambda x: x.iloc[0]['rw'] < 50)
    sessions_sub = pd.unique(df['session#'])
    array_switchProbs = []
    for i in range(len(sessions_sub)):
        session = sessions_sub[i]   
        print('session: ', session)
        switchProbs = []
        trials = 2
        df_sub = df[(df['session#']==session)]
        # if df_sub['rewprobfull1'].iloc[0]>df_sub['rewprobfull2'].iloc[0]:
        #     best = 1
        # else:
        #     best = 2
        
        # if start == 'correct': 
        #     df_sub = df[(df['session#']==session)&(df['port'].iloc[0] == best)]
        # elif start == 'incorrect':
        #     df_sub = df[(df['session#']==session)&(df['port'].iloc[0] != best)]
        # elif start == 'all':
        #     df_sub = df[(df['session#']==session)]
            
        # print(df_sub.head(2))
       
        # for e in range(sess_len):
        #     sum_switches = np.sum((df_sub['switch'].head(trials)).tolist())
        #     switch_prob = sum_switches/trials
        #     switch_prob = 
        #     switchProbs.append(switch_prob)
        #     trials += 1

        switchProbs = df_sub['switch'].rolling(window, center = True).mean(numeric_only= True)
        
        array_switchProbs.append(switchProbs)
    av_switchProbs = np.mean(array_switchProbs, axis = 0)
    
    return av_switchProbs
    # fig, ax = plt.subplots(layout = 'tight')
    # ax.plot(range(sess_len), av_switchProbs)
    # ax.set_xlabel('trial#')
    # ax.set_ylabel('switch probability')
    # # ax.set_ylim(0, 0.1)
    # plt.title(f'{animal}')
    # sns.despine()
    # plt.show()
    # return


In [64]:
# condition = 'easy'
# df_list = [subset_difference(df_str, condition), subset_difference(df_ustr, condition)]
df_list = [df_str, df_ustr]
# df_list = [buffalord, aggro, auroma]
fig, ax = plt.subplots(layout = 'tight')
c = 0
colors = ['g', 'r']
# colors = ['g', 'r', 'r']
for df in df_list:
    av_switchProbs = switchProb_trial(df, 149, 1000,  animal = '', start = 'incorrect')
    print(av_switchProbs)
    ax.plot(range(len(av_switchProbs)), av_switchProbs, colors[c])
    ax.set_xlabel('trial#')
    ax.set_ylabel('switch probability')
    # ax.set_ylim(0, 0.1)"
    # plt.title(f'{animal}')
    ax.set(ylim = [0, 0.45])
    sns.despine()
    plt.show()
    c+=1



no. of sess before subsetting:  1000
No. of sess after subsetting:  1000
session:  3
session:  7
session:  9
session:  15
session:  16
session:  20
session:  21
session:  24
session:  30
session:  31
session:  32
session:  33
session:  34
session:  36
session:  40
session:  41
session:  42
session:  43
session:  44
session:  46
session:  47
session:  49
session:  50
session:  53
session:  54
session:  55
session:  57
session:  59
session:  63
session:  64
session:  66
session:  67
session:  69
session:  71
session:  72
session:  78
session:  80
session:  81
session:  83
session:  86
session:  89
session:  90
session:  95
session:  96
session:  97
session:  98
session:  99
session:  100
session:  103
session:  104
session:  105
session:  108
session:  109
session:  112
session:  117
session:  118
session:  119
session:  120
session:  121
session:  128
session:  129
session:  130
session:  133
session:  136
session:  137
session:  140
session:  146
session:  149
session:  150
session:  1

#### cprobs

In [89]:
history_len = 3
conditional_probs1 = get_conditional_probs(df_str, history_len)
conditional_probs2 = get_conditional_probs(df_ustr, history_len)


In [93]:
conditional_probs1 = conditional_probs1.sort_values(by = 'pswitch')
conditional_probs2 = sort_cprobs(conditional_probs2, conditional_probs1['history'].values)

In [96]:
probs_list = [conditional_probs1, conditional_probs2]
color = ['g', 'r']

c=0
for probs in probs_list:
    fig, ax = plt.subplots(layout = 'tight', figsize = (12, 3))

    ax.bar(x = probs['history'], height = probs['pswitch'], alpha = 0.7, color = color[c])
    ax.set(xlabel = 'history', ylabel = ' pswitch')
    sns.despine()
    c+=1

In [5]:
choice_str(df_str, 50, 1)


In [4]:
df_str

,trial#,session#,port,reward,rewprobfull1,rewprobfull2,rw
0,1,1,1,0,60,40,60
1,2,1,1,0,60,40,60
2,3,1,2,1,60,40,40
3,4,1,2,0,60,40,40
4,5,1,1,0,60,40,60
...,...,...,...,...,...,...,...
9995,9996,50,2,1,20,80,80
9996,9997,50,2,1,20,80,80
9997,9998,50,2,0,20,80,80
9998,9999,50,2,1,20,80,80


In [67]:
conditional_probs1['history'].values

array(['AAA', 'aAA', 'AaA', 'AAa', 'aaA', 'aAa', 'Aaa', 'AbA', 'aBB',
       'abA', 'ABA', 'aaa', 'ABB', 'abB', 'Aba', 'aBb', 'AbB', 'aaB',
       'aBA', 'aba', 'ABb', 'ABa', 'abb', 'AaB', 'aAB', 'aab', 'Abb',
       'aBa', 'AAB', 'Aab', 'aAb', 'AAb'], dtype=object)

In [70]:
sort_cprobs(conditional_probs1, conditional_probs1['history'].values)


,history,pswitch,n,pswitch_err
0,AAA,0.026992,54571,0.000694
16,aAA,0.059250,13249,0.002051
8,AaA,0.065178,13793,0.002102
2,AAa,0.071394,14259,0.002156
24,aaA,0.125183,4777,0.004788
18,aAa,0.140959,5193,0.004829
10,Aaa,0.148995,5423,0.004835
12,AbA,0.170221,2720,0.007206
21,aBB,0.171522,3364,0.006499
28,abA,0.247995,1621,0.010726


In [6]:
bias_plot(df_ustr)


##### Plot specific combinations:

In [65]:
comb_list = ['aaa', 'aab', 'aba', 'abb']
fig, ax = plt.subplots(2, 2, figsize = (12, 6), layout = 'tight')
fig.suptitle('Group by choice history')
loc = [(0, 0), (0, 1), (1, 0), (1,1)] #list of (row, column) tuples
i = 0
for comb in comb_list:
    row, column = loc[i][0], loc[i][1] 
    #store indices of each combination:
    comb_str = cprobs_str_lower[cprobs_str_lower== comb].index.tolist()
    comb_ustr = cprobs_ustr_lower[cprobs_ustr_lower== comb].index.tolist()
    #calculate standard error
    err_str = stats.sem([cprobs_str_list[0][comb_str], cprobs_str_list[1][comb_str]])
    err_ustr = stats.sem([cprobs_ustr_list[0][comb_ustr], cprobs_ustr_list[1][comb_ustr]]) 
    #plot
    df_str = cprobs_str_df
    df_ustr = cprobs_ustr_df
    
    ax[row][column].bar(x =df_str[(df_str['history'].map(lambda x : x.lower())) == comb].history , height = df_str[(df_str['history'].map(lambda x : x.lower())) == comb].pswitch, color = 'g', alpha = 0.5, yerr = err_str)
    ax[row][column].bar(x = df_ustr[(df_ustr['history'].map(lambda x : x.lower())) == comb].history, height = df_ustr[(df_ustr['history'].map(lambda x : x.lower())) == comb].pswitch, color = 'r', alpha = 0.5, yerr = err_ustr)
    ax[row][column].set_ylim(bottom = 0)
    ax[row][column].title.set_text(comb_list[i].upper())
    i+=1


NameError: name 'cprobs_str_lower' is not defined

##### Add equal rewprobs to the task?

In [20]:
probs = np.delete(np.arange(0.1, 1, 0.1), 4)
p1, p2 = [], []
for i in range(50000):
    p1.append(np.random.choice(probs))
    if np.random.random() < 0.125:
        p2.append(p1[i])
    else:
        p2.append(1 - p1[i])

In [55]:
test_df = pd.DataFrame(data = {'p1':p1, 'p2': p2})
prob1, prob2 = 0.6, 0.4
((test_df[((test_df['p1']==prob1)&(test_df['p2']==prob2))|((test_df['p1']==prob2)&(test_df['p2']==prob1))]))

,p1,p2
0,0.6,0.4
10,0.6,0.4
18,0.6,0.4
20,0.4,0.6
23,0.4,0.6
...,...,...
49980,0.4,0.6
49987,0.4,0.6
49990,0.6,0.4
49993,0.4,0.6


In [58]:
probs = np.delete(np.arange(0.1, 1, 0.1), 4)
p1, p2 = [], []
for i in range(50000):
    p1.append(np.random.choice(probs))
    p2.append(np.random.choice(probs))
test_df2 = pd.DataFrame(data = {'p1':p1, 'p2':p2})
len(test_df2[test_df2['p1']==test_df2['p2']])

6307

In [ ]:
#have 0.5, 0.5 OR random probability of having equal rewprobs

In [13]:
#bias
bias_plot(df_str)